In [1]:
import sys
import subprocess


subprocess.run([sys.executable, "-m", "pip", "install", "pandas", "numpy", "scikit-learn"])

print("Instalación completada. Ahora importa las librerías:")
import pandas as pd
import numpy as np
print(f"Pandas {pd.__version__} y NumPy {np.__version__} instalados correctamente")

Instalación completada. Ahora importa las librerías:
Pandas 2.3.3 y NumPy 2.3.5 instalados correctamente


In [2]:
%%capture
!curl -L -o positive.csv https://raw.githubusercontent.com/Gavroshe/RuTweetCorp/master/positive.csv
!curl -L -o negative.csv https://raw.githubusercontent.com/Gavroshe/RuTweetCorp/master/negative.csv

In [10]:
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [6]:
import os
import pandas as pd

# Verifica el directorio actual
print("Directorio actual:", os.getcwd())
print("Archivos aquí:", os.listdir('.'))



positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)


negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)

df = pd.concat([positive, negative]) #This united the function -

Directorio actual: c:\Users\Usuario\Videos\Innovation Web\Project\analis_emotion\src
Archivos aquí: ['Emotions.ipynb', 'negative.csv', 'positive.csv']


In [7]:
print(df.head())

                                                text     label
0  @first_timee хоть я и школота, но поверь, у на...  positive
1  Да, все-таки он немного похож на него. Но мой ...  positive
2  RT @KatiaCheh: Ну ты идиотка) я испугалась за ...  positive
3  RT @digger2912: "Кто то в углу сидит и погибае...  positive
4  @irina_dyshkant Вот что значит страшилка :D\nН...  positive


In [8]:
df.sample(5, random_state=40)

,text,label
15931,RT @Blawar_1337: Теперь у нас с @Wake_UA появи...,positive
59532,с днём рождения зайка*))) ухх погуляем мы сего...,positive
47185,RT @Shumkova0406199: @ann_safina Вов вов вов А...,negative
42002,"Надо выдернуть звуковую дорожку из ""Доктора Ка...",positive
109035,@_hassliebe_ может все таки на этой неделе вер...,negative


We divide the data into training and test samples using the function - Разбиваем данные на обучающую и тесторовую выборки с помощью функции

In [11]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)


print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(170125,) (56709,) (170125,) (56709,)


In [12]:
y_train[:5]

90071    negative
59757    negative
35642    positive
94533    positive
90759    negative
Name: label, dtype: object

In [42]:
x_train[:5]

90071    RT @palnom6: Дождь в январе, #Вконтакте не раб...
59757    почему, когда хочется любить искренне?.. НЕ ПО...
35642    @HelFleur только это не Киев, наверное ) в Кие...
94533    наш город всегда стоит) Погода лишь немного ус...
90759    @drunktwi ну боян же вековой, тви фу таким быть((
Name: text, dtype: object

In [13]:
y_train.value_counts()

label
positive    86290
negative    83835
Name: count, dtype: int64

#Baseline: classification of unprocessed n-grams
We will try to get the transformation of sentences into a numerical vector.
Мы попробуем получить преобразование предложений в численный вектор.


In [18]:
!pip3 install nltk

In [22]:
import sys
!{sys.executable} -m pip install nltk --user

"c:\Users\Usuario\Videos\Innovation" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [23]:
import subprocess
import sys

# Instalar usando Python directamente (sin rutas con espacios)
result = subprocess.run([sys.executable, "-m", "pip", "install", "nltk"], 
                       capture_output=True, text=True)

if result.returncode == 0:
    print("✅ NLTK instalado correctamente")
else:
    print("⚠️  Intentando método alternativo...")
    # Método alternativo
    import pip
    pip.main(['install', 'nltk'])

✅ NLTK instalado correctamente


Функция для работы с n-граммами реализована в библиотке nltk (Natural Language ToolKit), импортируем эту функцию:

In [27]:
import nltk
from nltk import ngrams


In [53]:
sentence = 'Если б мне платили каждый раз'.split()
sentence

['Если', 'б', 'мне', 'платили', 'каждый', 'раз']

Чтобы получить n-грамму для такой последовательности, используем функцию ngrams().

In [54]:
list(ngrams(sentence, 1))#униграммы

[('Если',), ('б',), ('мне',), ('платили',), ('каждый',), ('раз',)]

In [55]:
list(ngrams(sentence, 2))#биграммы

[('Если', 'б'),
 ('б', 'мне'),
 ('мне', 'платили'),
 ('платили', 'каждый'),
 ('каждый', 'раз')]

In [56]:
list(ngrams(sentence, 3))#триграммы

[('Если', 'б', 'мне'),
 ('б', 'мне', 'платили'),
 ('мне', 'платили', 'каждый'),
 ('платили', 'каждый', 'раз')]

# Векторизация - Vectorization

In [48]:
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer # модель "мешка слов", см. далее

Настраиваем векторизатор - We configured the vectorizer

In [57]:
vectorizer = CountVectorizer(ngram_range=(1,1))

In [58]:
vectorized_x_train = vectorizer.fit_transform(x_train)

In [59]:
list(vectorizer.vocabulary_.items())[:10]

[('rt', 74611),
 ('palnom6', 67724),
 ('дождь', 126985),
 ('январе', 242796),
 ('вконтакте', 112973),
 ('не', 165452),
 ('работает', 199475),
 ('жизнь', 131688),
 ('боль', 107353),
 ('почему', 191483)]

После инициализации vectorizer можно обучить на наших данных.

В нашей выборке 170125 текстов (твитов), в них встречается 243835 разных слов.

In [60]:
vectorized_x_train.shape

(170125, 243835)

In [61]:
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(vectorized_x_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [62]:
vectorized_x_test = vectorizer.transform(x_test)

получаем результат анализа однограммы - we obtain the result of the unigram analysis

In [63]:
pred = clf.predict(vectorized_x_test)

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       0.76      0.76      0.76     28088
    positive       0.77      0.76      0.77     28621

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709

